In [ ]:
# from google.colab import files
# files.upload()

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,recall_score,precision_score

In [ ]:
df_train=pd.read_csv('../input/hackerearth-machine-learning-exhibit-art/dataset/train.csv')
df_test=pd.read_csv('../input/hackerearth-machine-learning-exhibit-art/dataset/test.csv')
df_train.head()

In [ ]:
df_train.shape

In [ ]:
df_train['Cost'].unique()

In [ ]:
df_train['Cost']=df_train['Cost'].abs()

In [ ]:
df_train['Cost']=np.log(df_train['Cost'])

In [ ]:
df_train['Scheduled Date']=pd.to_datetime(df_train['Scheduled Date'],format='%m/%d/%y')
df_train['Delivery Date']=pd.to_datetime(df_train['Delivery Date'],format='%m/%d/%y')

In [ ]:
df_train['S_month']=df_train['Scheduled Date'].dt.month
df_train['S_year']=df_train['Scheduled Date'].dt.year
df_train['S_Day']=df_train['Scheduled Date'].dt.day

In [ ]:
df_train['D_month']=df_train['Delivery Date'].dt.month
df_train['D_year']=df_train['Delivery Date'].dt.year
df_train['D_Day']=df_train['Delivery Date'].dt.day

In [ ]:
df_train.drop(['Scheduled Date','Delivery Date'],1,inplace=True)

In [ ]:
df_train.head()

In [ ]:
df_train['Transport'].fillna("missingTransport",inplace=True)
df_train['Material'].fillna("missingMaterial",inplace=True)
df_train['Remote Location'].fillna("missingRemoteLocation",inplace=True)

In [ ]:
df_train['Artist Reputation'].fillna(value=df_train['Artist Reputation'].mean(), inplace=True)
df_train['Height'].fillna(value=df_train['Height'].mean(), inplace=True)
df_train['Weight'].fillna(value=df_train['Weight'].mean(), inplace=True)
df_train['Width'].fillna(value=df_train['Width'].mean(), inplace=True)

In [ ]:
df_train.isnull().sum()

In [ ]:
y=df_train.Cost
X=df_train.drop(['Cost'],1)

In [ ]:
df_text=X[['Artist Name','Material','International','Express Shipment','Installation Included','Customer Location','Remote Location','Customer Information','Fragile','Transport']]

In [ ]:
df_text.head()

In [ ]:
df_num = X.drop(['Artist Name','Material','International','Express Shipment','Installation Included','Customer Location','Remote Location','Customer Information','Fragile','Transport'],1)

In [ ]:
df_num.head()

In [ ]:

df_num.drop('Customer Id',1,inplace=True)

In [ ]:


df_num.head()

In [ ]:
df_text['main'] = df_text[df_text.columns[0:]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1)

In [ ]:
df_text.main[1]

In [ ]:
X_text=df_text.main

In [ ]:
import regex as re
model = []
for i in range(0, df_train.shape[0]):
    data = re.sub(r'\W', ' ', str(X_text[i]))
    data = data.lower()
    data = re.sub(r'^br$', ' ', data)
    data = re.sub(r'\s+br\s+',' ',data)
    data = re.sub(r'\s+[a-z]\s+', ' ',data)
    data = re.sub(r'^b\s+', '', data)
    data = re.sub(r'\s+', ' ', data)
    model.append(data)   

In [ ]:
model[1]

In [ ]:
model[4]

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vector = TfidfVectorizer(max_features = 25, min_df = 1, max_df = 0.99, stop_words = stopwords.words('english'))
X_text = vector.fit_transform(model).toarray()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc=MinMaxScaler()


In [ ]:
df_num=sc.fit_transform(df_num,1)

In [ ]:
df_num

In [ ]:
X_num=df_num

In [ ]:
X=np.concatenate((X_num,X_text),1)

In [ ]:
X1 = pd.DataFrame(X)

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(n_jobs = -1, random_state = 42, n_estimators = 500, verbose = 1)
regr.fit(X_train, y_train)
regr.score(X_train,y_train)

In [ ]:
from sklearn import metrics
pred1 = regr.predict(X_test) 
rmse = np.sqrt(metrics.mean_squared_error(y_test, pred1)) 
print("RMSE : % f" %(rmse)) 

In [ ]:
metrics.r2_score(y_test,pred1)*100

In [ ]:
import lightgbm as lgb
train = lgb.Dataset(X_train, label=y_train)
test = lgb.Dataset(X_test, label=y_test)

In [ ]:
param = {'objective': 'regression',
         'boosting': 'gbdt',  
         'metric': 'l2_root',
         'learning_rate': 0.01, 
         'num_iterations': 10000,
         'min_data_in_leaf': 15,
         'bagging_fraction': 0.90,
         'bagging_freq': 2,
         'bagging_seed': 3,
         'feature_fraction': 0.99,
         'feature_fraction_seed': 2,
         'early_stopping_round': 2000,
         'max_bin': 500
         }

In [ ]:
gbm = lgb.LGBMRegressor(**param)

In [ ]:
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='l1',
        early_stopping_rounds = 100)

In [ ]:
pred1 = gbm.predict(X_test, num_iteration = gbm.best_iteration_) 
rmse = np.sqrt(metrics.mean_squared_error(y_test, pred1)) 
print("RMSE : % f" %(rmse)) 
print(metrics.r2_score(y_test,pred1)*100)

In [ ]:
df = pd.read_csv("../input/hackerearth-machine-learning-exhibit-art/dataset/test.csv")

In [ ]:
df1 = df[['Customer Id']]

In [ ]:
df.head(2)


In [ ]:
df['Scheduled Date'] = pd.to_datetime(df['Scheduled Date'],format='%m/%d/%y')

In [ ]:
df['S_month'] = df['Scheduled Date'].dt.month
df['S_year'] = df['Scheduled Date'].dt.year
df['S_day'] = df['Scheduled Date'].dt.day

In [ ]:
df['Delivery Date'] = pd.to_datetime(df['Delivery Date'],format='%m/%d/%y')

In [ ]:
df['D_month'] = df['Delivery Date'].dt.month
df['D_year'] = df['Delivery Date'].dt.year
df['D_day'] = df['Delivery Date'].dt.day

In [ ]:
df = df.drop(['Scheduled Date','Delivery Date'],1)

In [ ]:
df['Transport'].fillna("missingTransport",inplace=True)
df['Material'].fillna("missingMaterial",inplace=True)
df['Remote Location'].fillna("missingRemoteLocation",inplace=True)

In [ ]:
df['Artist Reputation'].fillna(value=df['Artist Reputation'].mean(), inplace=True)
df['Height'].fillna(value=df['Height'].mean(), inplace=True)
df['Weight'].fillna(value=df['Weight'].mean(), inplace=True)
df['Width'].fillna(value=df['Width'].mean(), inplace=True)

In [ ]:

df = df.drop(['Customer Id'],1)

In [ ]:
df_text = df[['Artist Name','Material','International','Express Shipment','Installation Included','Customer Location','Remote Location','Customer Information','Fragile','Transport']]

In [ ]:
df_num = df.drop(['Artist Name','Material','International','Express Shipment','Installation Included','Customer Location','Remote Location','Customer Information','Fragile','Transport'],1)

In [ ]:
df_text['main'] = df_text[df_text.columns[0:]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1)

df_text.main.head(3)

In [ ]:
X_text = df_text.main

In [ ]:
import regex as re
model = []
for i in range(0, df.shape[0]):
    data = re.sub(r'\W', ' ', str(X_text[i]))
    data = data.lower()
    data = re.sub(r'^br$', ' ', data)
    data = re.sub(r'\s+br\s+',' ',data)
    data = re.sub(r'\s+[a-z]\s+', ' ',data)
    data = re.sub(r'^b\s+', '', data)
    data = re.sub(r'\s+', ' ', data)
    model.append(data)   

In [ ]:
X_text = vector.transform(model).toarray()

In [ ]:
df_num = sc.transform(df_num)
df_num

In [ ]:
X_num = df_num

In [ ]:

X = np.concatenate((X_text,X_num),axis = 1)
X

In [ ]:
X_xg = pd.DataFrame(X)

In [ ]:
pred = gbm.predict(X)

In [ ]:

pred

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import KFold
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_log_error

errlgb = []
y_pred_totlgb = []

fold = KFold(n_splits = 10, shuffle=True, random_state=42)

for train_index, test_index in fold.split(X1):
    X_train, X_test = X1.loc[train_index], X1.loc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    lgbm = LGBMRegressor(**param)
    lgbm.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=0, early_stopping_rounds=200)

    y_pred_lgbm = lgbm.predict(X_test)
    print("RMSE LGBM: ",  np.sqrt(metrics.mean_squared_error(y_test, y_pred_lgbm)))
    print("R2 LGBM: ",  metrics.r2_score(y_test,y_pred_lgbm)*100)
    print("**********************")
    errlgb.append(np.sqrt(metrics.mean_squared_error(y_test, y_pred_lgbm)))
    p = lgbm.predict(X)
    y_pred_totlgb.append(p)

In [ ]:
np.mean(errlgb,0) #0.2829093072516393

In [ ]:
lgbm_final = np.mean(y_pred_totlgb,0)
lgbm_final = np.exp(lgbm_final)
lgbm_final

In [ ]:
data = {
        'Customer Id': df1['Customer Id'],
        'Cost': lgbm_final} 

In [ ]:
df_sub = pd.DataFrame(data = data)

In [ ]:
df_sub.to_csv('result_hk.csv',index=False)

In [ ]:
df_sub